In [2]:
!pip3 install folium

  Using cached folium-0.17.0-py2.py3-none-any.whl (108 kB)
  Using cached branca-0.7.2-py3-none-any.whl (25 kB)
  Using cached xyzservices-2024.6.0-py3-none-any.whl (83 kB)


In [3]:
import pandas as pd
import boto3
import zipfile
import glob
import gzip
import shutil

import folium
import os
from tqdm import tqdm

# 1. S3 연결

In [3]:
# s3 client
s3 = boto3.client('s3', 
                aws_access_key_id='id',
                aws_secret_access_key='key')

# 2. S3에 있는 데이터 확인 및 삭제

## 2.1. 데이터 확인

In [4]:
# 원하는 bucket과 하위경로에 있는 object list 출력
bucket_name = 'airdna-prod-data-integration'

obj_list = s3.list_objects(Bucket = bucket_name, Prefix = 'yanolja/outbound/June 2024')

In [6]:
# object list의 Content 리턴
contents_list = obj_list['Contents']
file_list = []
for content in contents_list:
    key = content['Key']
    file_list.append(key)
file_list

['yanolja/outbound/June 2024/',
 'yanolja/outbound/June 2024/Yanolja-PPD-South Korea-CSV_monthly_file_2024_07_07_02_29_51.csv',
 'yanolja/outbound/June 2024/Yanolja-PPD-South Korea-CSV_property_file_2024_07_07_02_29_48.csv']

## 2.1. 데이터 삭제

In [ ]:
# delete_list = []
# delete_list = file_list
# for dl in delete_list:
#     s3.delete_object(Bucket = bucket_name, Key = dl)


# 3. S3에서 데이터 불러오기 및 다운로드

### 해당 폴더 내 파일 확인
- airdna에서 파일을 주는 형태가 월별로 다름

In [6]:
month_list = ['January 2023', 'February 2023', 'March 2023', 'April 2023',
             'May 2023', 'June 2023', 'July 2023', 'August 2023', 'September 2023',
             'October 2023', 'November 2023', 'December 2023', 'January 2024', 'February 2024', 'March 2024', 'April 2024', 'May 2024', 'June 2024']

In [17]:
bucket_name = 'airdna-prod-data-integration'
    
def airdna_data_download(ml = None):
    prefix = 'yanolja/outbound/' + ml
    response = s3.list_objects_v2(Bucket = bucket_name, Prefix = prefix)
    
    if 'Contents' in response:
        directory_path = '../data/' + ml
        if not os.path.exists(directory_path):
            os.makedirs(directory_path)
            print(f"Directory '{directory_path}' created.")
        else:
            print(f"Directory '{directory_path}' already exists.")
        
        files = []
        for obj in response['Contents']:
            files.append(obj['Key'])
        print(files)
        for i in range(1, len(files)):
            object_key = files[i]
            local_file_path = directory_path + '/' + object_key.split('/')[3]
            #s3.download_file(bucket_name, object_key, local_file_path)
            print(object_key)
    else:
        print("No objects found in the specified bucket and prefix.")
        

In [20]:
s3.download_file(bucket_name, 'yanolja/outbound/June 2024/Yanolja-PPD-South Korea-CSV_monthly_file_2024_07_07_02_29_51.csv',
                '../data/June 2024/Yanolja-PPD-South Korea-CSV_monthly_file_2024_07_07_02_29_51.csv')


In [16]:
s3.download_file(bucket_name, 'yanolja/outbound/June 2024/Yanolja-PPD-South Korea-CSV_property_file_2024_07_07_02_29_48.csv',
                '../data/June 2024/Yanolja-PPD-South Korea-CSV_property_file_2024_07_07_02_29_48.csv')

In [18]:
airdna_data_download(ml = 'June 2024')

Directory '../data/June 2024' already exists.
['yanolja/outbound/June 2024/', 'yanolja/outbound/June 2024/Yanolja-PPD-South Korea-CSV_monthly_file_2024_07_07_02_29_51.csv', 'yanolja/outbound/June 2024/Yanolja-PPD-South Korea-CSV_property_file_2024_07_07_02_29_48.csv']
yanolja/outbound/June 2024/Yanolja-PPD-South Korea-CSV_monthly_file_2024_07_07_02_29_51.csv
yanolja/outbound/June 2024/Yanolja-PPD-South Korea-CSV_property_file_2024_07_07_02_29_48.csv


### 파일 다운로드
- bucket & 폴더 내 파일을 모두 다운로드

In [ ]:
# def unzip_(zip_path = None, unzip_path = None):
#     with zipfile.ZipFile(zip_path, 'r') as zip:
#         zip.extractall(unzip_path)

In [ ]:
# def ungzip_(zip_path = None, unzip_path = None):
#     with gzip.open(zip_path, 'rt') as f:
#         df = pd.read_csv(f, sep = ',')
#     return df

In [ ]:
# df_property = ungzip_(zip_path = '../data/March 2024/Yanolja-PPD-South Korea-CSV_property_file_2024_02_08_17_59_53.csv.gz', 
#              unzip_path = 'file.csv')

# df_performance = ungzip_(zip_path = '../data/March 2024/Yanolja-PPD-South Korea-CSV_monthly_file_2024_02_08_17_59_55.csv.gz', 
#              unzip_path = 'file.csv')


# 3. 데이터 로딩

In [31]:
# df_property = pd.read_csv('../data/March 2024/Yanolja-PPD-South Korea-CSV_property_file_2024_04_06_03_16_49.csv')

In [243]:
df_property.iloc[0]

Property ID                        abnb_601800677259491116
Airbnb Property ID                    601800677259491072.0
Airbnb HOST ID                                 357045305.0
VRBO Property ID                                       NaN
VRBO Host ID                                           NaN
                                            ...           
Revenue Potential LTM (Native)                 12069259.18
Cleaning Fee (Native)                             43253.28
Published Nightly Rate (Native)                  458214.43
Published Monthly Rate (Native)                        NaN
Published Weekly Rate (Native)                         NaN
Name: 0, Length: 80, dtype: object

In [ ]:
df_property.iloc[]

In [4]:
df_performance = pd.read_csv('../data/June 2024/Yanolja-PPD-South Korea-CSV_monthly_file_2024_07_07_02_29_51.csv')
df_performance['month'] = pd.to_datetime(df_performance['Reporting Month'])

df_performance['reporting_year'] = df_performance['Reporting Month'].str[:4]


/tmp/ipykernel_124/1401477627.py:1: DtypeWarning: Columns (18,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_performance = pd.read_csv('../data/June 2024/Yanolja-PPD-South Korea-CSV_monthly_file_2024_07_07_02_29_51.csv')


In [5]:
df_performance.shape

(8482691, 34)

In [6]:
df_performance.columns

Index(['Property ID', 'Property Type', 'Listing Type', 'Bedrooms',
       'Reporting Month', 'Occupancy Rate', 'Currency', 'Revenue (USD)',
       'Revenue Potential (USD)', 'ADR (USD)', 'Number of Reservations',
       'Reservation Days', 'Available Days', 'Blocked Days', 'Country',
       'State', 'City', 'Zipcode', 'Neighborhood',
       'Metropolitan Statistical Area', 'Latitude', 'Longitude', 'Active',
       'Scraped During Month', 'Airbnb Property ID', 'Airbnb Host ID',
       'Vrbo Property ID', 'Vrbo Host ID', 'Property Manager',
       'Revenue (Native)', 'ADR (Native)', 'Revenue Potential (Native)',
       'month', 'reporting_year'],
      dtype='object')

In [7]:
df_performance['City'].unique()

array(['Seoul', 'Jeju Island', nan, 'Jeollanam-do', 'Gyeongju-si',
       'Busan', 'Incheon', 'Seongnam', 'Gangwon-do', 'Jeollabuk-do',
       'Goyang', 'Daegu', 'Daejeon', 'Chungcheongbuk-do', 'Gwangju',
       'Suwon', 'Yongin', 'Ulsan', 'Gyeonggi-do', 'Bucheon', 'Anyang',
       'Chungcheongnam-do', 'Paengseong-eup', 'Uijeongbu', 'Chiang Mai'],
      dtype=object)

In [21]:
def abnb_listing(data = None, year =None):
    
    subset = data[(data['Active'] == True) & (data['Available Days'] > 0) & (data['reporting_year'] == year)]
    subset = subset[['Property ID', 'Occupancy Rate', 'Revenue (USD)', 'ADR (USD)', 'Number of Reservations', 'Reservation Days',
                'Available Days', 'Blocked Days', 'State', 'City', 'Active', 'Revenue (Native)', 'ADR (Native)', 'reporting_year', 'Reporting Month']]
    subset['adr(krw)'] = subset['Revenue (Native)'] / subset['Reservation Days'] 
    subset['adr(krw)'] = subset['adr(krw)'].fillna(0)
    
    lower_bound = subset['adr(krw)'].quantile(0.02)
    upper_bound = subset['adr(krw)'].quantile(0.98)

    # Filter the DataFrame
    subset = subset[(subset['adr(krw)'] > lower_bound) & (subset['adr(krw)'] < upper_bound)]

    
    return subset
    
def abnb_listing(data = None, month =None):
    
    subset = data[(data['Active'] == True) & (data['Available Days'] > 0) & (data['Reporting Month'] == month)]
    subset = subset[['Property ID', 'Occupancy Rate', 'Revenue (USD)', 'ADR (USD)', 'Number of Reservations', 'Reservation Days',
                'Available Days', 'Blocked Days', 'State', 'City', 'Active', 'Revenue (Native)', 'ADR (Native)', 'reporting_year', 'Reporting Month',
                    'month']]
    subset['adr(krw)'] = subset['Revenue (Native)'] / subset['Reservation Days'] 
    subset['adr(krw)'] = subset['adr(krw)'].fillna(0)
    
    lower_bound = subset['adr(krw)'].quantile(0.02)
    upper_bound = subset['adr(krw)'].quantile(0.98)

    # Filter the DataFrame
    subset = subset[(subset['adr(krw)'] > lower_bound) & (subset['adr(krw)'] < upper_bound)]

    
    return subset
    


In [22]:
df_performance['Reporting Month'].iloc[:3]

0    2024-01-01
1    2024-02-01
2    2024-03-01
Name: Reporting Month, dtype: object

In [23]:
result = abnb_listing(data = df_performance, month = '2024-01-01')

In [28]:
result['Property ID'].nunique()

66822

In [30]:
i

'2024-01-01'

In [32]:
temp = abnb_listing(data = df_performance, month = i)

In [40]:
temp['Revenue (Native)'].sum()

90491186299.29001

In [41]:
abnb_listing_korea = []
for i in tqdm(df_performance['Reporting Month'].unique()):
    abnb_listing_korea.append([i, abnb_listing(data = df_performance, month = i)['Property ID'].nunique(),  abnb_listing(data = df_performance, month = i)['Revenue (Native)'].sum()])

100%|██████████| 121/121 [02:57<00:00,  1.46s/it]


In [43]:
pd.DataFrame(abnb_listing_korea).to_csv('ddd.csv')

In [25]:
result['Property ID'].nunique()

66822

In [32]:
def abnb_performance(data = None, month = None):
    occ_result = []
    adr_result = []
    revpar_result = []
    subset = data[(data['Active'] == True) & (data['Available Days'] > 0) & (data['Reporting Month'] == month)]
    subset = subset[['Property ID', 'Occupancy Rate', 'Revenue (USD)', 'ADR (USD)', 'Number of Reservations', 'Reservation Days',
                'Available Days', 'Blocked Days', 'State', 'City', 'Active', 'Revenue (Native)', 'ADR (Native)']]

    subset = subset[subset['City'].notna()]


#     subset['occ'] = subset['Reservation Days'] / (31 - subset['Blocked Days'])
    subset['occ'] = subset['Occupancy Rate']
    subset['adr(usd)'] = subset['Revenue (USD)'] / subset['Reservation Days']
    subset['adr(krw)'] = subset['Revenue (Native)'] / subset['Reservation Days']
    subset['adr(krw)'] = subset['adr(krw)'].fillna(0)
    
    lower_bound = subset['adr(krw)'].quantile(0.02)
    upper_bound = subset['adr(krw)'].quantile(0.98)

    # Filter the DataFrame
    subset = subset[(subset['adr(krw)'] > lower_bound) & (subset['adr(krw)'] < upper_bound)]

    
    subset['revpar'] = subset['occ'] * subset['adr(krw)']
    
    subset['City'].unique()
    region_dict = {'Gangwon-do': '강원', 'Jeju Island': '제주', 'Seoul': '서울', 'Daegu': '경북권',
                  'Busan': '부산', 'Gyeongju-si': '경남권', 'Suwon': '경기권', 'Anyang': '경기권',
                  'Chungcheongbuk-do': '충청권', 'Incheon': '경기권', 'Daejeon': '충청권', 'Gwangju': '전라권',
                  'Gyeonggi-do': '경기권', 'Seongnam': '경기권', 'Paengseong-eup': '경기권', 'Bucheon': '경기권',
                  'Jeollanam-do': '전라권', 'Jeollabuk-do': '전라권', 'Yongin': '경기권', 'Chungcheongnam-do': '충청권',
                  'Ulsan': '경남권', 'Goyang': '경기권', 'Uijeongbu': '경기권'}

    subset['Region'] = subset['City'].replace(region_dict)
    subset = subset.reset_index()
    for rg in subset['Region'].unique():
        occ_result.append([rg, subset[subset['Region'] == rg]['occ'].mean()])
        adr_result.append([rg, subset[subset['Region'] == rg]['adr(krw)'].mean()])
        revpar_result.append([rg, subset[subset['Region'] == rg]['revpar'].mean()])
    occ_result.append(['전국', subset['occ'].mean()])
    adr_result.append(['전국', subset['adr(krw)'].mean()])
    revpar_result.append(['전국', subset['revpar'].mean()])
    occ_df = pd.DataFrame(occ_result).transpose()
    adr_df = pd.DataFrame(adr_result).transpose()
    revpar_df = pd.DataFrame(revpar_result).transpose()
    
    occ_df.columns = occ_df.iloc[0]
    adr_df.columns = adr_df.iloc[0]
    revpar_df.columns = revpar_df.iloc[0]
    
    occ_df = occ_df[1:]
    adr_df = adr_df[1:]
    revpar_df = revpar_df[1:]
    
    col_order = ['전국', '강원', '경기권', '경남권', '경북권', '부산', '서울', '전라권', '제주', '충청권']
    occ_df = occ_df.reindex(columns = col_order)
    adr_df = adr_df.reindex(columns = col_order)
    revpar_df = revpar_df.reindex(columns = col_order)
    return occ_df, adr_df, revpar_df

In [34]:
occ_final = []

periods = ['2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01','2023-05-01','2023-06-01',
          '2023-07-01','2023-08-01','2023-09-01','2023-10-01','2023-11-01','2023-12-01',
          '2024-01-01','2024-02-01','2024-03-01', '2024-04-01', '2024-05-01', '2024-06-01']
for i in tqdm(periods):
    occ_final.append(abnb_performance(data = df_performance, month = i)[0])

occ_final = pd.concat(occ_final, ignore_index = True)
occ_final['periods'] = periods
occ_final.to_csv('abnb_occ.csv', encoding = 'cp949')

100%|██████████| 18/18 [00:17<00:00,  1.00it/s]


In [35]:
adr_final = []

periods = ['2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01','2023-05-01','2023-06-01',
          '2023-07-01','2023-08-01','2023-09-01','2023-10-01','2023-11-01','2023-12-01',
          '2024-01-01','2024-02-01','2024-03-01', '2024-04-01', '2024-05-01', '2024-06-01']
for i in tqdm(periods):
    adr_final.append(abnb_performance(data = df_performance, month = i)[1])

adr_final = pd.concat(adr_final, ignore_index = True)
adr_final['periods'] = periods
adr_final.to_csv('abnb_adr.csv', encoding = 'cp949')

100%|██████████| 18/18 [00:17<00:00,  1.00it/s]


In [36]:
revpar_final = []

periods = ['2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01','2023-05-01','2023-06-01',
          '2023-07-01','2023-08-01','2023-09-01','2023-10-01','2023-11-01','2023-12-01',
          '2024-01-01','2024-02-01','2024-03-01', '2024-04-01', '2024-05-01', '2024-06-01']
for i in tqdm(periods):
    revpar_final.append(abnb_performance(data = df_performance, month = i)[2])

revpar_final = pd.concat(revpar_final, ignore_index = True)
revpar_final['periods'] = periods
revpar_final.to_csv('abnb_revpar.csv', encoding = 'cp949')

100%|██████████| 18/18 [00:17<00:00,  1.01it/s]
